# 05 动态切片策略与重叠机制提升RAG召回率
- 本讲重点：探讨主流RAG框架（LangChain, LlamaIndex, Dify）如何结合动态切片与重叠机制提升召回能力。


---

## 碎片化原因
- **连贯性丧失：** 语义中断，大模型理解受损。
- **相关性稀释：** 关键信息被“稀释”，检索排名下降。
- **信息分散：** 多跳推理信息不全，答案不完整。
- **结果：** “垃圾进，垃圾出”，增**“幻觉”**风险。

---

## 策略：动态切片 + 重叠机制

### 动态切片：源头避免碎片化
- **定义：** 智能、自适应切分，根据语义、结构、主题定切分点。
- **类型：**
    - **内容感知型：**
        - **语义切片：** 识别语义断点。
        - **主题切片：** 按主题切分。
        - **优势：** 减少语义割裂，提高连贯性。
    - **结构感知型：**
        - **布局感知切片：** 利用PDF、HTML结构元素。
        - **文档特异性切片：** 针对特定格式（如Markdown）。
        - **优势：** 保留文档结构。
    - **高级自适应型：**
        - **智能体切片：** LLM判断切分边界。
        - **优势：** 更灵活智能。

### 重叠机制：非动态策略缓冲
- **核心：** 相邻区块保留重复内容，形成“滑动窗口”。
- **作用：**
    - 局部语义保持。
    - 缓解信息缺失。
    - 提升召回率。
- **建议：** 区块大小的 **10%-20%**。

---

## 协同效果
- 动态切片缓解碎片化，重叠机制作补充，增强上下文完整性。

---

## 最佳实践
- **结构化文档：** 优先结构感知切片。
- **纯文本：** 推荐语义切片（如LlamaIndex的`SemanticSplitterNodeParser`）。
- **尝试：** 父子模式。
- **重叠机制：** 必要时启用，控制在 **10%-20%**。
- **持续优化：** 建立评估体系。

## 固定大小文本切块（递归方法）及其改进

### 主要问题
- **上下文割裂：** 机械截断文本，破坏语义。
- **语义完整性受损：** 区块内句意不完整，影响匹配精度和大模型回答质量。

### 改进策略
- **引入重叠机制（Overlap）：** 相邻块保留重复内容（如50字符），确保连贯性。
- **智能截断：** 尽量在标点符号或段落结束处切分。
### 实践工具：LangChain 的 RecursiveCharacterTextSplitter

In [ ]:
! pip install langchain llama-index 
! pip install llama-index-llms-openai llamaindex-py-client
! pip install langchain-text-splitters

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=5,
    length_function=len,
    separators=["\n", ".", ""],
)

text = """
Why does chunking cause context fragmentation?

1. Loss of Coherence
When a complete semantic unit is forcibly split, the information becomes incomplete
(the meaning is broken). For example, if an argument is distributed across two chunks,
neither chunk alone can accurately convey the original meaning. This interferes with
the language model’s understanding and generation, leading to incomplete or even
misleading outputs.

2. Diluted Relevance
If a chunk mixes relevant and irrelevant content, the key information becomes diluted.
This negatively affects the accuracy of vector representations and, in turn, lowers
retrieval ranking performance.

3. Scattered Information
For complex questions that require multi-hop reasoning, relevant information may be
scattered across multiple chunks. If not all of them are retrieved, a RAG system cannot
produce a complete answer.

When these issues compound, they directly lead to a “garbage in, garbage out” effect,
and may even increase the risk of model hallucinations.
"""

# Text to be processed
documents = text_splitter.create_documents([text])

for doc in documents:
    print(doc)

page_content='Why does chunking cause context fragmentation?'
page_content='1. Loss of Coherence'
page_content='When a complete semantic unit is forcibly split,'
page_content='lit, the information becomes incomplete'
page_content='(the meaning is broken)'
page_content='. For example, if an argument is distributed acros'
page_content='across two chunks,'
page_content='neither chunk alone can accurately convey the ori'
page_content='e original meaning'
page_content='. This interferes with'
page_content='the language model’s understanding and generation'
page_content='ation, leading to incomplete or even'
page_content='misleading outputs.

2. Diluted Relevance'
page_content='If a chunk mixes relevant and irrelevant content,'
page_content='tent, the key information becomes diluted'
page_content='.'
page_content='This negatively affects the accuracy of vector re'
page_content='or representations and, in turn, lowers'
page_content='retrieval ranking performance.'
page_content='3. Scattered I

### 实践工具：LangChain 的 RecursiveCharacterTextSplitter
- **核心优势：** 利用一组优先级分割符递归切分文本，尽可能保留语义完整。
- **工作原理：**
    - **初步切分：** 用第一个分隔符（如`\n`）段落划分。
    - **检查长度：** 若某段超`chunk_size`，用下一个分隔符（如`。`）切分。
    - **递归处理：** 依次尝试剩余分隔符，直到满足长度。
    - **合并优化：** 相邻小块合并后仍小于`chunk_size`，则合并。

### 切块策略建议
- **根据内容类型选择：**
    - **逻辑紧密型：** 尽量保持段落完整性。
    - **语义独立型：** 可按句子切分。
- **结合向量化模型特性：**
    - 模型对长文本处理不佳：适当缩短块长。
    - 模型擅长短文本：可适度切分但保留关键上下文。
- **关注LLM输入限制：** 控制块长，避免超模型最大输入。
- **持续实验与优化：** 无普适最佳实践，需测试验证，建立评估体系。

---

## 表1：切片策略对RAG指标的量化影响（综合视图）

| 切片策略   | 关键评估指标 / 发现与结果                                                                                                                                                                             | 来源 / 备注                                         | 适用场景                               |
| :--------- | :---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------- | :------------------------------------- |
| **页面级切片** | **端到端答案准确率**：达到最高的平均准确率 (0.648)，性能最稳定。                                                                                                                                | NVIDIA研究                                          | 结构化、多页商业文档                   |
| **语义切片 (Spacy)** | **上下文相关性、答案相关性、忠实度**：处理不同难度查询时，总体表现优于递归等方法。                                                                                                            | Antematter研究                                      | 富含语义信息的非结构化文本             |
| **令牌级切片** | **答案不正确率、平均倒数排名(MRR)**：<br>- 300-500个令牌的区块大小是性能“甜点区”，更大区块性能下降。<br>- 检索数量K=4是延迟与性能的最佳平衡点。 | Arize AI研究                                        | 通用文本切片，需关注模型输入限制       |
| **递归字符切片** | **上下文相关性**：表现不佳，无法有效处理文本间的上下文相似性。                                                                                                                                  | Antematter研究                                      | 基础策略，但易破坏语义连贯性             |


- **核心洞察：** 不存在普适“最佳”切片策略，最优选择取决于**文档类型**和**查询复杂性**。
- **趋势：** 先进生产级RAG系统需**动态切片路由器**，根据文档特征选择策略（如PDF用布局解析器，.txt用语义分割器，.py用代码分割器）。

---

## 重叠机制的作用与代价

### 核心作用
- **维护指代关系：** 边界处维持局部语义连贯。
- **提升检索准确性：** 增强跨区块信息关联，提高召回率和匹配质量。

### 成本与挑战
- **存储成本增加：** 向量数据库体积膨胀，索引构建时间延长。
- **计算开销上升：** 索引规模增大，增加向量搜索负担，延长查询延迟。
- **冗余信息传递：** 大量重叠区块会浪费LLM上下文窗口，不增新信息。
- **建议：** 重叠设置为 `chunk_size` 的 **10%-20%**，通过实验调整。


---

## 动态切块：LlamaIndex 的 SemanticSplitterNodeParser

固定大小的文本切块在RAG系统中常导致 **“上下文碎片化”**，影响检索和生成质量。为解决此问题，LlamaIndex推出了**SemanticSplitterNodeParser**，这是一款通过语义理解实现智能切分的先进工具。

### 核心机制
SemanticSplitterNodeParser的核心思想是：**根据文本语义变化点进行动态切分**，而非依赖固定字符数或句法结构。

其工作流程如下：
- **句子级拆分：** 将文档按句子进行划分。
- **组块构建：** 连续多个句子组合成一个“句子组”（由`buffer_size`控制）。
- **语义嵌入生成：** 使用指定嵌入模型（如OpenAI或HuggingFace模型）为每个句子组生成向量表示。
- **语义相似度计算：** 通过余弦距离衡量相邻句子组之间的语义差异。
- **动态切分决策：** 当语义差异超过设定的阈值（由`breakpoint_percentile_threshold`控制），则在此处插入切分点。

这种方式确保了每个文本块内部**语义连贯、信息完整**，极大提升了后续检索与生成的效果。

### 核心参数解析
- **`embed_model`** (BaseEmbedding, 必需):
    - 用于生成语义向量的嵌入模型。这是语义比较的基础，其质量直接决定切片效果。
- **`buffer_size`** (整数, 默认: 1):
    - 评估语义相似性时，组合在一起的句子数量。
    - 设为1表示逐句比较；大于1则将多个句子视为一个单元，有助于考虑更广泛的上下文。
- **`breakpoint_percentile_threshold`** (整数, 默认: 95):
    - 确定切分点的余弦距离百分位阈值。
    - **敏感度调节参数：** 较低值（如80）意味着对语义变化更敏感，产生更多、更小的区块；较高值（如98）要求语义变化非常显著才切分，产生更少、更大的区块。

### 代码实现示例

In [ ]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.schema import Document
import os

# Set API key
# os.getenv("OPENAI_API_KEY")

# Example text containing multiple topics
multi_theme_text = (
    "Artificial intelligence (AI) is fundamentally transforming the healthcare industry. "
    "AI algorithms can analyze medical images and diagnose diseases such as cancer earlier "
    "and more accurately than human radiologists. "
    "In addition, AI plays a critical role in drug discovery by predicting the effectiveness "
    "of chemical compounds, significantly reducing the time required to bring new drugs to market. "
    "Shifting gears, let us talk about financial technology (FinTech). "
    "Mobile payments have become mainstream worldwide, with digital wallets and contactless "
    "payments reshaping consumer behavior. "
    "Blockchain technology provides decentralized solutions for cross-border payments and "
    "asset tokenization, with the potential to reshape the underlying infrastructure of the "
    "entire financial system."
)

# Create a document object
document = Document(text=multi_theme_text)

# Initialize embedding model
embed_model = OpenAIEmbedding()

# Initialize semantic splitter
splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=90,
    embed_model=embed_model,
)

# Perform semantic splitting
nodes = splitter.get_nodes_from_documents([document])

# Print results
print(f"Semantic splitting produced {len(nodes)} nodes:")
for i, node in enumerate(nodes):
    print(f"--- Node {i + 1} ---")
    print(node.get_content())
    print("-" * 20)


语义切分后得到 1 个节点：
--- 节点 1 ---
人工智能（AI）正在彻底改变医疗保健行业。AI算法能够通过分析医学影像，比人类放射科医生更早、更准确地诊断出癌症等疾病。此外，AI在药物研发中也发挥着关键作用，能够预测化合物的有效性，从而大大缩短新药上市的时间。话锋一转，我们来谈谈金融科技（FinTech）。移动支付已经成为全球主流，数字钱包和非接触式支付改变了人们的消费习惯。区块链技术则为跨境支付和资产代币化提供了去中心化的解决方案，有望重塑整个金融体系的底层架构。
--------------------


---

## 实践建议

### 1. 嵌入模型选择
- **通用文本：** 选`OpenAI`的`text-embedding-ada-002`。
- **专业领域：** 用**领域预训练模型**（如`BioBERT`）或**微调模型**。
- **注意：** 嵌入模型质量直接影响切分效果。

### 2. `buffer_size`设置
- **`buffer_size=1`：** 逐句比较，适合语义边界明显文本。
- **`buffer_size>1`：** 适合语义渐变的长段落。

### 3. 切分敏感度调节
- **高敏感度（低阈值，如80）：** 适用于精细切分，如多跳问答。
- **低敏感度（高阈值，如98）：** 适用于保留完整段落。

---

`SemanticSplitterNodeParser`是LlamaIndex中实现语义切片的核心，通过语义理解避免上下文断裂。

### 推荐策略
- **优先**高质量、领域适配的**嵌入模型**。
- **合理设置**`buffer_size`和`breakpoint_percentile_threshold`。
- **借助可视化**工具优化。

**效果：** 通过语义驱动的动态切片，显著提升RAG系统召回率与生成质量，尤其适用于复杂文本。


## 超越句子：使用命题（Propositions）进行原子化检索

传统切片难满足RAG高精度与完整上下文需求。LlamaIndex引入**“命题”（Propositions）原子化检索**。
- 体现“由小到大”思想：细粒度检索提高准确率，结合父区块提供完整上下文。
- 目标：协同提升检索精度与生成质量。

### 什么是“命题”？
- **定义：** 文本中**原子化的、自包含的事实单元**（《Dense X Retrieval》论文）。
- **特征：**
    - **不可再分：** 不能拆解为更小语义单位。
    - **独立表达：** 脱离上下文独立解释事实/概念。
    - **自然语言形式：** 简洁明了，无需额外信息。
- **示例：** “埃菲尔铁塔位于巴黎，建于1889年。”可提取为：
    - “埃菲尔铁塔位于巴黎”
    - “埃菲尔铁塔建于1889年”
- **作用：** 高精确命题作检索单元，命中后追溯原始父区块，提供LLM完整上下文，确保生成内容准确完整。

### LlamaIndex 中的命题化实现方案

#### 1. TopicNodeParser：LLM驱动的主题一致性重组器（代价高）
- **功能：**
    - 利用LLM将段落分解为多个命题。
    - 根据主题一致性重组命题，形成新语义区块。
- **参数：** `similarity_method`（LLM或嵌入模型判断主题相关性）。
- **场景：** 检索精度要求极高、允许高计算成本的场景。
- **优势：** 精准捕捉语义边界，深度解析复杂文本。
- **代价：** 依赖大模型，推理成本高，适合离线处理。

#### 2. DenseXRetrievalPack：开箱即用的命题化解决方案
- **核心流程：**
    - 自动为知识库节点提取命题。
    - 构建专门针对命题的检索器。
    - 返回与命题相关的原始父区块用于生成。
- **优势：** 快速部署命题化检索，无需手动编写提示词或训练。
- **架构：** 结合LLM提取命题 + 向量索引构建 + 递归检索。

### 命题提取的实现细节
- LlamaIndex和Langchain均有实现。
- LlamaIndex使用论文提供的提示词生成命题。

---

In [ ]:
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.packs.dense_x_retrieval import DenseXRetrievalPack
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.llama_pack import download_llama_pack
import os
import nest_asyncio
import json

# Apply nest_asyncio to support async calls (e.g., in notebooks)
nest_asyncio.apply()


PROPOSITIONS_PROMPT = PromptTemplate(
    """Decompose the "Content" into clear and simple propositions, ensuring they are interpretable out of
context.
1. Split compound sentence into simple sentences. Maintain the original phrasing from the input
whenever possible.
2. For any named entity that is accompanied by additional descriptive information, separate this
information into its own distinct proposition.
3. Decontextualize the proposition by adding necessary modifier to nouns or entire sentences
and replacing pronouns (e.g., "it", "he", "she", "they", "this", "that") with the full name of the
entities they refer to.
4. Present the results as a list of strings, formatted in JSON.

Input: Title: ¯Eostre. Section: Theories and interpretations, Connection to Easter Hares. Content:
The earliest evidence for the Easter Hare (Osterhase) was recorded in south-west Germany in
1678 by the professor of medicine Georg Franck von Franckenau, but it remained unknown in
other parts of Germany until the 18th century. Scholar Richard Sermon writes that "hares were
frequently seen in gardens in spring, and thus may have served as a convenient explanation for the
origin of the colored eggs hidden there for children. Alternatively, there is a European tradition
that hares laid eggs, since a hare’s scratch or form and a lapwing’s nest look very similar, and
both occur on grassland and are first seen in the spring. In the nineteenth century the influence
of Easter cards, toys, and books was to make the Easter Hare/Rabbit popular throughout Europe.
German immigrants then exported the custom to Britain and America where it evolved into the
Easter Bunny."
Output: [ "The earliest evidence for the Easter Hare was recorded in south-west Germany in
1678 by Georg Franck von Franckenau.", "Georg Franck von Franckenau was a professor of
medicine.", "The evidence for the Easter Hare remained unknown in other parts of Germany until
the 18th century.", "Richard Sermon was a scholar.", "Richard Sermon writes a hypothesis about
the possible explanation for the connection between hares and the tradition during Easter", "Hares
were frequently seen in gardens in spring.", "Hares may have served as a convenient explanation
for the origin of the colored eggs hidden in gardens for children.", "There is a European tradition
that hares laid eggs.", "A hare’s scratch or form and a lapwing’s nest look very similar.", "Both
hares and lapwing’s nests occur on grassland and are first seen in the spring.", "In the nineteenth
century the influence of Easter cards, toys, and books was to make the Easter Hare/Rabbit popular
throughout Europe.", "German immigrants exported the custom of the Easter Hare/Rabbit to
Britain and America.", "The custom of the Easter Hare/Rabbit evolved into the Easter Bunny in
Britain and America." ]

Input: {node_text}
Output:"""
)


def safe_json_loads(text: str) -> list:
    """Safely parse a JSON list from LLM output (handles ```json fenced blocks)."""
    # Strip whitespace and invisible characters
    text = text.strip()

    # If it starts with ```json, remove the code fence marker
    if text.startswith("```json"):
        text = text[7:]
    if text.endswith("```"):
        text = text[:-3]

    # Strip again
    text = text.strip()

    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError at position {e.pos}: {e}")
        return []


def extract_propositions(text: str, llm: OpenAI) -> list[str]:
    """
    Use an LLM to extract propositions from the input text.
    """
    # Build the full prompt
    prompt = PROPOSITIONS_PROMPT.format(node_text=text)

    # Call the LLM
    response = llm.complete(prompt).text.strip()

    # Parse the response as a JSON list
    try:
        propositions = safe_json_loads(response)
    except Exception:
        print("Failed to parse JSON. Raw response:", response)
        return []

    return propositions


# Initialize the LLM + embedding model
llm = OpenAI(model="gpt-4o", temperature=0.1, max_tokens=750)
embed_model = OpenAIEmbedding(embed_batch_size=128)

# Example text for testing extract_propositions
test_text = "The Eiffel Tower is located in Paris and was built in 1889."

# Run and print results
propositions = extract_propositions(test_text, llm)
print("Extracted propositions:")
print(propositions)


提取出的命题：
['The Eiffel Tower is located in Paris.', 'The Eiffel Tower was built in 1889.']


In [ ]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.llama_pack import download_llama_pack
import nest_asyncio
nest_asyncio.apply()

# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY"

# DenseXRetrievalPack = download_llama_pack(
#     "DenseXRetrievalPack", "./dense_pack"
# )

# If you have already downloaded DenseXRetrievalPack, you can import it directly.
from llama_index.packs.dense_x_retrieval import DenseXRetrievalPack

# Load documents
dir_path = "/Users/wilson/rag50_test"
documents = SimpleDirectoryReader(dir_path).load_data()

# Use LLM to extract propositions from every document/node

dense_pack = DenseXRetrievalPack(documents)

response = dense_pack.run("When the Eiffel Tower was built?")
print(response)


Generating embeddings: 100%|██████████| 11/11 [00:02<00:00,  4.95it/s]


The Eiffel Tower was built in 1889.


---

## 内部原理详解

`DenseXRetrievalPack` 的核心逻辑遵循**“由小到大”（small-to-big）**的检索思想：

### 步骤一：基础切块（Nodes）
- 使用 `SentenceSplitter` 将文档划分为**基本文本块（nodes）**，作为命题提取的**基础单元**。

### 步骤二：命题提取（Sub-Nodes）
- 调用 **LLM** 和**预定义Prompt**，异步提取每个 `node` 中的**命题**，转化为**子节点（sub-nodes）**。
- 同时保留 `sub-node` 与原始 `node` 的**映射关系**。

### 步骤三：构建混合索引
- 将原始 `nodes` 与 `sub-nodes` 一起构建**向量索引（VectorStoreIndex）**。
- 支持**命题级检索**（高精度）与**区块级生成**（完整上下文）。

### 步骤四：递归检索机制
- 使用 `RecursiveRetriever` 进行检索：
    - **优先检索子命题：** 确保高精度匹配。
    - **未找到则回溯父区块：** 保证信息完整性。

这种索引结构兼顾了**检索效率**与**上下文完整性**。

---

## 命题化检索的价值

命题化检索是RAG领域的前沿方向：
- **突破传统限制：** 不再局限于段落或句子作为最小单位。
- **双重提升：** 通过精细化切片和上下文增强机制，同时提升**检索精度**与**生成质量**。
- **适用场景：** 特别适合**知识密集型任务**，如多跳问答、法律文书分析、科研文献检索等。

LlamaIndex提供了从`SentenceSplitter` → `SemanticSplitter` → `TopicNodeParser` → `DenseXRetrievalPack`的**“复杂度阶梯”**，让开发者可以根据项目需求和性能预算灵活选择策略，并能随业务增长平滑升级RAG架构。

---

接下来，我们将探讨商业RAG软件如何解决这些问题。


## Dify 实践：可视化配置下的智能切片策略

- **Dify：** 以用户体验为核心的LLM应用平台。
- **切片模式：**
    - **通用模式：** 扁平化文本块，适用于简单文本。
    - **父子模式：** 推荐新用户，实现“由小到大”层次化检索。
- **注意：** 知识库创建后，切片模式不可更改。

### 父子模式：平衡检索精度与上下文完整性
- **工作机制：**
    - **分层切分：** 文档先切为“父区块”，再细分“子区块”。
    - **精确检索：** 用子区块匹配，提升准确率。
    - **上下文增强：** 找到子区块后，提取其父区块作为上下文。
- **可控性强：** 可分别设置父子区块切分规则。
- **优势特性：** 结构化解耦重叠需求，子区块无需手动设置重叠。

---

## RAGFlow 实践：深度文档理解驱动的切片方案

- **RAGFlow：** 企业级复杂格式文档RAG引擎，强调 **“布局感知”** 切片。
- **核心设计哲学：**
    - 传统切片忽略文档视觉结构，RAGFlow融合CV与NLP实现 **“深度文档理解”** 。
    - **目标：** 高保真还原结构，避免格式错误导致的上下文断裂。
    - **原则：** “高质量输入，高质量输出”。

### 模板化切片：布局感知的标准化路径
- **机制：** 根据文档类型预设优化方案，自动应用解析逻辑。

| 模板名称       | 适用场景          | 特点                                          |
| :------------- | :---------------- | :-------------------------------------------- |
| General        | 简单文本文档      | 标准连续切片                                  |
| Paper          | 学术论文          | 识别标题、摘要、正文、参考文献               |
| Book           | 长篇幅书籍        | 保留章节结构，支持跨页连贯性                  |
| Laws           | 法律条文          | 处理编号条款、嵌套结构                        |
| Table          | 含有表格的文档    | 解析表格结构并保留表头信息                    |
| Resume         | 简历文件          | 提取姓名、联系方式、工作经历等                |
| One            | 短文档或全局上下文 | 整个文档视为一个整体                          |
| Picture        | 图片中含文字内容  | OCR结合NLP分析图像内文本                      |

- **核心价值：** 自动应用优化规则，尤其在PDF等复杂版式中显著优势。
- **用户操作：**
    - **可视化审查：** 查看文档快照及区块内容。
    - **手动微调：** 编辑区块内容，添加关键词。
- **设计理念：** **自动化 + 人工协同**。

---

## 总结与最佳实践建议：实现动态切片与重叠机制的高效协同

### 1. 从结构入手：优先使用结构感知策略
- **场景：** 明确格式文档（HTML, PDF等）。
- **做法：** RAGFlow模板化、LlamaIndex的HTMLNodeParser/TableNodeParser。
- **优势：** 保留原始语义与结构，提升检索质量。

### 2. 对纯文本，走向语义：采用语义驱动切片
- **场景：** 非结构化文本（新闻、报告等）。
- **做法：** LlamaIndex的`SemanticSplitterNodeParser`。
- **优势：** 提升上下文连贯性，精准检索复杂语义。

### 3. 拥抱“由小到大”架构：探索层次化检索模式
- **场景：** 需兼顾精度与完整性任务（多跳问答）。
- **做法：** Dify父子模式、LlamaIndex的`SentenceWindowNodeParser`、命题化检索。
- **优势：** 小块精度，大块完整性；规避重叠依赖。

### 4. 将重叠视为战术补充：适度使用而非依赖
- **场景：** 固定/递归切片时的辅助手段。
- **做法：** 10%-20%重叠比例，避免冗余。
- **优势：** 简单策略下提升上下文，成本可控。

### 5. 构建评估体系：持续验证与迭代优化
- **场景：** 任何RAG应用上线及运行中。
- **做法：** 定义指标（召回率、准确性），A/B测试，结合人工反馈。
- **优势：** 避免盲目选择，持续优化，适应需求。

---

- **切片作用：** 为混合检索、重排、查询转换、多阶段检索等高级技术提供高质量数据“弹药”。
- **目标：** 确保切片策略的输出能为整个RAG系统奠定坚实基础。
